In [1]:
import pandas as pd

In [3]:
df = pd.read_csv("./IndustryData.csv")
df

,Industry,Bank,Year,Rank,Net Revenue (Millions USD),Market %,No. of deals
0,TMT,Goldman Sachs,2020,1,1850,18.5,95
1,TMT,Morgan Stanley,2020,2,1620,16.2,88
2,TMT,JPMorgan,2020,3,1400,14.0,80
3,TMT,Goldman Sachs,2021,1,2450,20.1,130
4,TMT,Morgan Stanley,2021,2,2100,17.2,122
...,...,...,...,...,...,...,...
130,Financial Sponsors,JPMorgan,2023,2,1380,17.3,92
131,Financial Sponsors,Morgan Stanley,2023,3,1200,15.0,85
132,Financial Sponsors,Goldman Sachs,2024,1,1650,19.1,108
133,Financial Sponsors,JPMorgan,2024,2,1490,17.3,99


In [14]:
import plotly.graph_objects as go
import plotly.express as px

# Calculate total market share for each industry and year
industry_market_share = df.groupby(['Industry', 'Year']).apply(
    lambda x: x.iloc[0]['Net Revenue (Millions USD)'] / (x.iloc[0]['Market %'] / 100)
).reset_index(name='Total Market Share')

# Round the total market share to 2 decimal places for better readability
industry_market_share['Total Market Share'] = industry_market_share['Total Market Share'].round(2)

# Plot total market share for each industry year on year using bar graphs with trend lines
industries = industry_market_share['Industry'].unique()

fig = go.Figure()

for industry in industries:
    industry_data = industry_market_share[industry_market_share['Industry'] == industry]
    color = px.colors.qualitative.Plotly[industries.tolist().index(industry) % len(px.colors.qualitative.Plotly)]
    
    fig.add_trace(go.Bar(
        x=industry_data['Year'],
        y=industry_data['Total Market Share'],
        name=f'Total Market Share: {industry}',
        text=industry_data['Total Market Share'],
        textposition='outside',  # Position text outside for better visibility
        marker=dict(color=color, line=dict(color='black', width=1))  # Set bar color and add border
    ))
    fig.add_trace(go.Scatter(
        x=industry_data['Year'],
        y=industry_data['Total Market Share'],
        mode='lines+markers',
        name=f'Trend: {industry}',
        line=dict(color=color, width=2),  # Set trend line color to match bar color
        connectgaps=False  # Ensure the line only connects within the same industry
    ))

fig.update_layout(
    title='Total Market Share by Industry and Year',
    xaxis_title='Year',
    yaxis_title='Total Market Share (Millions USD)',
    barmode='group',
    template='plotly_white',  # Use a white theme for a professional look
    legend_title_text='Industry'
)

fig.show()


In [21]:

# Create a subplot of 3x3 for 9 industries
from plotly.subplots import make_subplots

fig = make_subplots(rows=3, cols=3, subplot_titles=industries[:9])

# Iterate over the first 9 industries to create individual plots
for i, industry in enumerate(industries[:9]):
    industry_data = industry_market_share[industry_market_share['Industry'] == industry]
    row = i // 3 + 1
    col = i % 3 + 1
    
    # Add vertical bar plot for each industry
    fig.add_trace(go.Bar(
        x=industry_data['Year'],
        y=industry_data['Total Market Share'],
        name=f'Total Market Share: {industry}',
        text=industry_data['Total Market Share'].round(2),  # Round to 2 decimals
        textposition='inside',  # Position text inside for a professional look
        textangle=0  # Ensure text is horizontal
    ), row=row, col=col)
    
    # Add trend line for each industry
    fig.add_trace(go.Scatter(
        x=industry_data['Year'],
        y=industry_data['Total Market Share'],
        mode='lines+markers',
        name=f'Trend: {industry}'
    ), row=row, col=col)

# Update layout for aesthetics
fig.update_layout(
    title='Total Market Share by Industry and Year (Top 9 Industries)',
    height=900,  # Adjust height for better visibility
    showlegend=False,  # Hide legend to avoid clutter
    template='plotly_white'  # Use a white theme for better visibility
)

# Update axes titles and ensure visibility of all years
fig.update_xaxes(title_text='Year', tickvals=[2020, 2021, 2022, 2023, 2024])
fig.update_yaxes(title_text='Total Market Share (Millions USD)')

fig.show()


In [11]:
# # Black BG


# # Create a subplot of 3x3 for 9 industries
# from plotly.subplots import make_subplots

# fig = make_subplots(rows=3, cols=3, subplot_titles=industries[:9])

# # Iterate over the first 9 industries to create individual plots
# for i, industry in enumerate(industries[:9]):
#     industry_data = industry_market_share[industry_market_share['Industry'] == industry]
#     row = i // 3 + 1
#     col = i % 3 + 1
    
#     # Add bar plot for each industry
#     fig.add_trace(go.Bar(
#         x=industry_data['Year'],
#         y=industry_data['Total Market Share'],
#         name=f'Total Market Share: {industry}',
#         text=industry_data['Total Market Share'],
#         textposition='auto'
#     ), row=row, col=col)
    
#     # Add trend line for each industry
#     fig.add_trace(go.Scatter(
#         x=industry_data['Year'],
#         y=industry_data['Total Market Share'],
#         mode='lines+markers',
#         name=f'Trend: {industry}'
#     ), row=row, col=col)

# # Update layout for aesthetics
# fig.update_layout(
#     title='Total Market Share by Industry and Year (Top 9 Industries)',
#     height=900,  # Adjust height for better visibility
#     showlegend=False,  # Hide legend to avoid clutter
#     template='plotly_dark'  # Use a dark theme for aesthetics
# )

# # Update axes titles
# fig.update_xaxes(title_text='Year')
# fig.update_yaxes(title_text='Total Market Share (Millions USD)')

# fig.show()



In [24]:
df.columns

Index(['Industry', 'Bank', 'Year', 'Rank', 'Net Revenue (Millions USD)',
       'Market %', 'No. of deals'],
      dtype='object')

In [42]:

# Iterate over each industry to create a separate figure for each
for industry in df['Industry'].unique():
    # Filter data for the current industry
    industry_data = df[df['Industry'] == industry]
    
    # Filter data for the top 3 banks as per the rank column within the industry
    top_banks_industry = industry_data[industry_data['Rank'] <= 3]
    
    # Group by bank and year, then calculate total net revenue and market share
    bank_summary_industry = top_banks_industry.groupby(['Bank', 'Year']).agg({
        'Net Revenue (Millions USD)': 'sum',
        'Market %': 'mean',
        'No. of deals': 'sum'
    }).rename(columns={
        'Net Revenue (Millions USD)': 'Net Revenue',
        'Market %': 'Market Share',
        'No. of deals': 'Deals'
    }).reset_index()
    
    # Market Share Comparison for the current industry
    fig_market_share = go.Figure()
    for bank in bank_summary_industry['Bank'].unique():
        bank_data = bank_summary_industry[bank_summary_industry['Bank'] == bank]
        fig_market_share.add_trace(go.Bar(
            x=bank_data['Year'],
            y=bank_data['Market Share'],
            name=f'{bank} Market Share'
        ))
    
    # Update layout for Market Share figure
    fig_market_share.update_layout(
        title=f'{industry} Market Share Comparison',
        height=400,
        showlegend=True,
        template='plotly_dark',
        xaxis_title='Year',
        yaxis_title='Market Share (%)',
        bargap=0.1  # Reduce gap between bars
    )
    
    fig_market_share.show()
    
    # Net Revenue Comparison for the current industry
    fig_net_revenue = go.Figure()
    for bank in bank_summary_industry['Bank'].unique():
        bank_data = bank_summary_industry[bank_summary_industry['Bank'] == bank]
        fig_net_revenue.add_trace(go.Bar(
            x=bank_data['Year'],
            y=bank_data['Net Revenue'],
            name=f'{bank} Net Revenue'
        ))

    # Update layout for Net Revenue figure
    fig_net_revenue.update_layout(
        title=f'{industry} Net Revenue Comparison',
        height=400,
        showlegend=True,
        template='plotly_dark',
        xaxis_title='Year',
        yaxis_title='Net Revenue (Millions USD)',
        bargap=0.1  # Reduce gap between bars
    )

    fig_net_revenue.show()


In [48]:
import pandas as pd
import plotly.express as px


df['Year'] = df['Year'].astype(int)
df['Rank'] = df['Rank'].astype(int)
df['Net Revenue (Millions USD)'] = pd.to_numeric(df['Net Revenue (Millions USD)'], errors='coerce')

# 3) Loop over industries
for industry in df['Industry'].unique():
    df_ind = df[df['Industry'] == industry]

    # 4) For each year, grab the top-3 banks by Rank
    top3 = (
        df_ind
        .sort_values(['Year', 'Rank'])
        .groupby('Year', as_index=False)
        .head(3)
        .dropna(subset=['Net Revenue (Millions USD)'])  # just in case
    )
    

    # 5) Build the bar chart
    fig = px.bar(
        top3,
        x='Year',
        y='Net Revenue (Millions USD)',
        color='Bank',
        barmode='group',
        title=f'Top 3 Banks in {industry} (2020–2024) by Net Revenue',
        template='plotly_white'
    )

    # 6) Tidy up spacing & axis labels
    fig.update_layout(
        title_x=0.5,
        xaxis_title='Year',
        yaxis_title='Net Revenue',
        bargap=0.15,       # gap between year-groups
        bargroupgap=0.05   # gap between banks within each year
    )

    # 7) Show it
    fig.show()


In [59]:
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# 1) Load & clean
# df = pd.read_csv('/mnt/data/IndustryData.csv')
df['Year'] = df['Year'].astype(int)
df['Rank'] = df['Rank'].astype(int)
df['Net Revenue (Millions USD)'] = pd.to_numeric(df['Net Revenue (Millions USD)'], errors='coerce')

# 2) Loop through industries
for industry in df['Industry'].unique():
    df_ind = df[df['Industry'] == industry]

    # 3) For each year, pick its top-3 banks by Rank
    top3 = (
        df_ind
        .sort_values(['Year','Rank'])
        .groupby('Year', as_index=False)
        .head(3)
        .dropna(subset=['Net Revenue (Millions USD)'])
    )

    years = sorted(top3['Year'].unique())
    n = len(years)

    # 4) Create n subplots side by side, sharing the Y-axis
    fig = make_subplots(
        rows=1, cols=n,
        shared_yaxes=True,
        subplot_titles=[str(y) for y in years]
    )

    # 5) Add one Bar trace per year-subplot
    for idx, year in enumerate(years, start=1):
        dfy = top3[top3['Year'] == year]
        fig.add_trace(
            go.Bar(
                x=dfy['Bank'],
                y=dfy['Net Revenue (Millions USD)'],
                text=dfy['Net Revenue (Millions USD)'],       # show values on bars
                textposition='outside'
            ),
            row=1, col=idx
        )

    # 6) Tidy up layout
    fig.update_layout(
        title_text=f"Top 3 Banks in {industry} (2020–2024) by Net Revenue",
        title_x=0.5,
        showlegend=False,
        height=400,
        margin=dict(t=80, b=40, l=60, r=20)
    )
    fig.update_yaxes(title_text="Net Revenue", row=1, col=1)
    for i in range(1, n+1):
        fig.update_xaxes(title_text="", row=1, col=i)

    fig.show()


In [65]:
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.colors import qualitative

# 1) Load & clean
# df = pd.read_csv('/mnt/data/IndustryData.csv')
df['Year'] = df['Year'].astype(int)
df['Rank'] = df['Rank'].astype(int)
df['Net Revenue (Millions USD)'] = pd.to_numeric(df['Net Revenue (Millions USD)'], errors='coerce')

# 2) Loop through industries
for industry in df['Industry'].unique():
    df_ind = df[df['Industry'] == industry]

    # 3) Get top-3 by rank per year
    top3 = (
        df_ind
        .sort_values(['Year', 'Rank'])
        .groupby('Year', as_index=False)
        .head(3)
        .dropna(subset=['Net Revenue (Millions USD)'])
    )

    years = sorted(top3['Year'].unique())
    n_years = len(years)

    # 4) Prepare a color map for all banks appearing in this industry
    all_banks = sorted(top3['Bank'].unique())
    palette   = qualitative.Plotly
    color_map = {b: palette[i % len(palette)] for i, b in enumerate(all_banks)}

    # 5) Create subplots (1 row × n_years cols), shared Y-axis
    fig = make_subplots(
        rows=1, cols=n_years,
        shared_yaxes=True,
        subplot_titles=[str(y) for y in years]
    )

    # 6) Add one trace per bank-in-year
    seen = set()  # to ensure each bank only shows up once in the legend
    for col, year in enumerate(years, start=1):
        dfy = top3[top3['Year'] == year]
        for _, row in dfy.iterrows():
            bank = row['Bank']
            rev  = row['Net Revenue (Millions USD)']
            fig.add_trace(
                go.Bar(
                    x=[bank],
                    y=[rev],
                    name=bank,
                    marker_color=color_map[bank],
                    showlegend=(bank not in seen),
                    text=[f"{rev:,}"],
                    textposition='outside'
                ),
                row=1, col=col
            )
            seen.add(bank)

    # 7) Tidy layout
    fig.update_layout(
        title_text=f"Top 3 Banks in {industry} (2020–2024) by Net Revenue",
        title_x=0.5,
        height=450,
        margin=dict(t=80, b=40, l=60, r=20),
        bargap=0.2
    )
    # Y-axis label only on first subplot
    fig.update_yaxes(title_text="Net Revenue", row=1, col=1)
    # Remove x-axis titles (bank names are the tick labels)
    for c in range(1, n_years+1):
        fig.update_xaxes(title_text="", row=1, col=c)

    fig.show()


In [66]:
import pandas as pd

# Assuming `top3` is the DataFrame containing the data
# Filter data for JPMorgan
jpm_data = top3[top3['Bank'] == 'JPMorgan']

# Calculate average net revenue for JPMorgan
avg_net_revenue_jpm = jpm_data['Net Revenue (Millions USD)'].mean()

# Calculate average market share for JPMorgan
avg_market_share_jpm = jpm_data['Market %'].mean()

# Calculate average number of deals for JPMorgan
avg_deals_jpm = jpm_data['No. of deals'].mean()

# Compare JPMorgan's performance with the average of all banks
avg_net_revenue_all = top3['Net Revenue (Millions USD)'].mean()
avg_market_share_all = top3['Market %'].mean()
avg_deals_all = top3['No. of deals'].mean()

# Insights
insights = {
    "JPMorgan Average Net Revenue": avg_net_revenue_jpm,
    "All Banks Average Net Revenue": avg_net_revenue_all,
    "JPMorgan Average Market Share": avg_market_share_jpm,
    "All Banks Average Market Share": avg_market_share_all,
    "JPMorgan Average Number of Deals": avg_deals_jpm,
    "All Banks Average Number of Deals": avg_deals_all
}

# Determine where JPMorgan is lacking
lacking_insights = {}
if avg_net_revenue_jpm < avg_net_revenue_all:
    lacking_insights['Net Revenue'] = "JPMorgan's average net revenue is below the industry average."
if avg_market_share_jpm < avg_market_share_all:
    lacking_insights['Market Share'] = "JPMorgan's average market share is below the industry average."
if avg_deals_jpm < avg_deals_all:
    lacking_insights['Number of Deals'] = "JPMorgan's average number of deals is below the industry average."

# Print insights
print("JPMorgan Performance Insights:")
for key, value in insights.items():
    print(f"{key}: {value}")

print("\nAreas where JPMorgan is lacking:")
for key, value in lacking_insights.items():
    print(f"{key}: {value}")


JPMorgan Performance Insights:
JPMorgan Average Net Revenue: 1734.0
All Banks Average Net Revenue: 1718.0
JPMorgan Average Market Share: 17.78
All Banks Average Market Share: 17.593333333333337
JPMorgan Average Number of Deals: 109.2
All Banks Average Number of Deals: 109.33333333333333

Areas where JPMorgan is lacking:
Number of Deals: JPMorgan's average number of deals is below the industry average.


In [68]:
import pandas as pd
import numpy as np

def analyze_jpm_performance(path_to_csv):
    # --- 1) Load & sanitize ---
    df = pd.read_csv(path_to_csv)
    df['Year'] = df['Year'].astype(int)
    df['Rank'] = df['Rank'].astype(int)
    df['Net Revenue (Millions USD)'] = pd.to_numeric(
        df['Net Revenue (Millions USD)'], errors='coerce'
    )
    
    # Ensure JPMorgan is present
    if 'JPMorgan' not in df['Bank'].unique():
        raise ValueError("No JPMorgan data found!")
    
    # --- 2) Prepare output rows ---
    rows = []
    
    # --- 3) Loop per industry ---
    for industry, grp in df.groupby('Industry'):
        # JPMorgan’s data for this industry
        jp = grp[grp['Bank'] == 'JPMorgan'].set_index('Year')
        all_banks = grp.set_index('Year')
        
        # Only consider years where JPM has data
        years = sorted(jp.index.intersection(all_banks.index))
        
        # a) Rank metrics
        avg_jp_rank  = jp.loc[years, 'Rank'].mean()
        med_ind_rank = (
            all_banks.loc[years]
            .groupby('Year')['Rank']
            .median()
            .mean()
        )
        rank_sd = jp.loc[years, 'Rank'].std()
        
        # b) Revenue CAGR helper
        def compute_cagr(series):
            series = series.sort_index()
            if len(series) < 2 or series.iloc[0] <= 0:
                return np.nan
            n_years = series.index.max() - series.index.min()
            return (series.iloc[-1] / series.iloc[0]) ** (1/n_years) - 1
        
        jp_rev     = jp['Net Revenue (Millions USD)'].dropna()
        ind_avg_rev = (
            all_banks.loc[jp_rev.index]
            .groupby('Year')['Net Revenue (Millions USD)']
            .mean()
        )
        
        jp_cagr  = compute_cagr(jp_rev)
        ind_cagr = compute_cagr(ind_avg_rev)
        
        # c) Collate
        rows.append({
            'Industry':       industry,
            'avg_jp_rank':    avg_jp_rank,
            'med_ind_rank':   med_ind_rank,
            'rank_sd':        rank_sd,
            'jp_cagr':        jp_cagr,
            'ind_cagr':       ind_cagr,
            'flag_rank':      avg_jp_rank > med_ind_rank,
            'flag_cagr':      jp_cagr < ind_cagr
        })
    
    summary = pd.DataFrame(rows)
    summary['lacking'] = summary['flag_rank'] | summary['flag_cagr']
    
    # --- 4) Display results ---
    print("\n=== JPMORGAN PERFORMANCE SUMMARY ===\n")
    print(summary[[
        'Industry','avg_jp_rank','med_ind_rank','rank_sd','jp_cagr','ind_cagr','lacking'
    ]].sort_values('lacking', ascending=False).to_string(index=False))
    
    print("\n=== ACTIONABLE INSIGHTS ===")
    for _, row in summary[summary['lacking']].iterrows():
        notes = []
        if row['flag_rank']:
            notes.append(
                f"avg rank {row['avg_jp_rank']:.2f} vs median {row['med_ind_rank']:.2f}"
            )
        if row['flag_cagr']:
            notes.append(
                f"CAGR {row['jp_cagr']:.1%} vs industry {row['ind_cagr']:.1%}"
            )
        print(f"– In **{row['Industry']}**, JPMorgan is underperforming ({'; '.join(notes)})")
    
    return summary

# Example usage:
# summary_df = analyze_jpm_performance('/mnt/data/IndustryData.csv')


In [70]:
analyze_jpm_performance('./IndustryData.csv')


=== JPMORGAN PERFORMANCE SUMMARY ===

          Industry  avg_jp_rank  med_ind_rank  rank_sd   jp_cagr  ind_cagr  lacking
 Consumer & Retail          3.0           2.0 0.000000 -0.027592 -0.036985     True
               FIG          1.2           2.0 0.447214 -0.038986 -0.036573     True
Financial Sponsors          2.0           2.0 0.000000 -0.039412 -0.036406     True
        Healthcare          1.2           2.0 0.447214 -0.029396 -0.027257     True
       Industrials          1.0           2.0 0.000000 -0.033820 -0.031453     True
       Real Estate          3.0           2.0 0.000000 -0.027402 -0.028872     True
               TMT          3.0           2.0 0.000000 -0.041839 -0.049201     True
            Energy          1.8           2.0 0.447214  0.028197  0.026988    False
 Natural Resources          2.0           2.0 0.707107  0.042369  0.014275    False

=== ACTIONABLE INSIGHTS ===
– In **Consumer & Retail**, JPMorgan is underperforming (avg rank 3.00 vs median 2.00)
– In 

,Industry,avg_jp_rank,med_ind_rank,rank_sd,jp_cagr,ind_cagr,flag_rank,flag_cagr,lacking
0,Consumer & Retail,3.0,2.0,0.000000,-0.027592,-0.036985,True,False,True
1,Energy,1.8,2.0,0.447214,0.028197,0.026988,False,False,False
2,FIG,1.2,2.0,0.447214,-0.038986,-0.036573,False,True,True
3,Financial Sponsors,2.0,2.0,0.000000,-0.039412,-0.036406,False,True,True
4,Healthcare,1.2,2.0,0.447214,-0.029396,-0.027257,False,True,True
5,Industrials,1.0,2.0,0.000000,-0.033820,-0.031453,False,True,True
6,Natural Resources,2.0,2.0,0.707107,0.042369,0.014275,False,False,False
7,Real Estate,3.0,2.0,0.000000,-0.027402,-0.028872,True,False,True
8,TMT,3.0,2.0,0.000000,-0.041839,-0.049201,True,False,True


In [ ]:
import pandas as pd
import streamlit as st
import plotly.express as px

# ————————————————————————————————————————————————
# 1) Load & clean data
# ————————————————————————————————————————————————
@st.cache_data
def load_data(path):
    df = pd.read_csv(path)
    df['Year'] = df['Year'].astype(int)
    df['Rank'] = df['Rank'].astype(int)
    df['Net Revenue (Millions USD)'] = pd.to_numeric(
        df['Net Revenue (Millions USD)'], errors='coerce'
    )
    return df

# df = load_data('/mnt/data/IndustryData.csv')

# ————————————————————————————————————————————————
# 2) Compute YoY growth
# ————————————————————————————————————————————————
# Pivot so we can pct_change per bank+industry
df_sorted = df.sort_values(['Industry','Bank','Year'])
df_sorted['YoY Growth (%)'] = (
    df_sorted
    .groupby(['Industry','Bank'])['Net Revenue (Millions USD)']
    .pct_change() * 100
)

# ————————————————————————————————————————————————
# 3) Sidebar selectors
# ————————————————————————————————————————————————
banks      = ['All Banks'] + sorted(df['Bank'].unique())
industries = ['All Industries'] + sorted(df['Industry'].unique())

sel_bank     = st.sidebar.selectbox("Select Bank", banks)
sel_ind      = st.sidebar.selectbox("Select Industry", industries)

# ————————————————————————————————————————————————
# 4) Filtered DataFrame
# ————————————————————————————————————————————————
mask = pd.Series(True, index=df_sorted.index)
if sel_bank != 'All Banks':
    mask &= df_sorted['Bank'] == sel_bank
if sel_ind != 'All Industries':
    mask &= df_sorted['Industry'] == sel_ind

df_filt = df_sorted[mask].dropna(subset=['YoY Growth (%)'])

st.title("Year-Over-Year Revenue Growth Analysis")
st.markdown(f"**Bank:** {sel_bank}  \n**Industry:** {sel_ind}")

# ————————————————————————————————————————————————
# 5) Show summary table
# ————————————————————————————————————————————————
st.subheader("YoY Growth Table")
st.dataframe(
    df_filt[['Industry','Bank','Year','Net Revenue (Millions USD)','YoY Growth (%)']]
    .reset_index(drop=True)
)

# ————————————————————————————————————————————————
# 6) Visualization
# ————————————————————————————————————————————————
st.subheader("YoY Growth Chart")

if sel_ind == 'All Industries':
    # line chart: one line per industry
    fig = px.line(
        df_filt,
        x='Year',
        y='YoY Growth (%)',
        color='Industry',
        markers=True,
        title="YoY % Growth by Industry"
    )
else:
    # bar chart for the single industry: one bar per year, colored by bank
    fig = px.bar(
        df_filt,
        x='Year',
        y='YoY Growth (%)',
        color='Bank',
        barmode='group',
        title=f"YoY % Growth in {sel_ind}"
    )

fig.update_layout(
    # To run this Streamlit code, save the notebook as a Python script (e.g., main.py) and execute the following command in your terminal:
    # streamlit run main.py
    title_x=0.5,
    xaxis=dict(dtick=1),
    yaxis_title="YoY Growth (%)"
)
st.plotly_chart(fig, use_container_width=True)


2025-04-27 02:09:24.196 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-04-27 02:09:26.298 
  command:

    streamlit run C:\Users\JAINIL\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()

In [74]:
import pandas as pd
import numpy as np
import streamlit as st
import plotly.express as px
import plotly.graph_objects as go

# ————————————————————————————————————————————————
# 1) Load & clean data
# ————————————————————————————————————————————————
@st.cache_data
def load_data(path):
    df = pd.read_csv(path)
    df['Year'] = df['Year'].astype(int)
    df['Rank'] = df['Rank'].astype(int)
    df['Net Revenue (Millions USD)'] = pd.to_numeric(
        df['Net Revenue (Millions USD)'], errors='coerce'
    )
    return df

# df = load_data('/mnt/data/IndustryData.csv')

# ————————————————————————————————————————————————
# 2) Precompute metrics
# ————————————————————————————————————————————————
# 2.1 Year-over-Year growth
df = df.sort_values(['Industry','Bank','Year'])
df['YoY Growth %'] = df.groupby(['Industry','Bank'])['Net Revenue (Millions USD)']\
                        .pct_change() * 100

# 2.2 CAGR helper
def compute_cagr(series):
    series = series.dropna().sort_index()
    if len(series) < 2 or series.iloc[0] <= 0:
        return np.nan
    n = series.index.max() - series.index.min()
    return (series.iloc[-1] / series.iloc[0]) ** (1/n) - 1

# 2.3 Build a summary table per Bank×Industry
summary_rows = []
for (bank, industry), grp in df.groupby(['Bank','Industry']):
    years = grp['Year'].unique()
    net = grp.set_index('Year')['Net Revenue (Millions USD)']
    yoy = grp.set_index('Year')['YoY Growth %']
    rank = grp.set_index('Year')['Rank']
    summary_rows.append({
        'Bank': bank,
        'Industry': industry,
        'CAGR %': compute_cagr(net) * 100,
        'Avg YoY %': yoy.mean(),
        'Rank Volatility': rank.std(),
        'Avg Rank': rank.mean()
    })
summary = pd.DataFrame(summary_rows)

# ————————————————————————————————————————————————
# 3) Sidebar controls
# ————————————————————————————————————————————————
st.sidebar.header("🔍 Filters & Options")
banks      = sorted(df['Bank'].unique())
industries = sorted(df['Industry'].unique())

sel_bank    = st.sidebar.selectbox("Bank", ["All Banks"] + banks)
sel_inds    = st.sidebar.multiselect("Industries",
                                     ["All Industries"] + industries,
                                     default=["All Industries"])
metric_tabs = st.sidebar.radio("Choose analysis", ["Overview", "Trends", "Heatmap"])

# interpret “All Industries”
if "All Industries" in sel_inds:
    filtered_inds = industries
else:
    filtered_inds = sel_inds

# slice data
df_f = df.copy()
if sel_bank != "All Banks":
    df_f = df_f[df_f['Bank'] == sel_bank]
df_f = df_f[df_f['Industry'].isin(filtered_inds)]
summary_f = summary[
    (summary['Bank'] == sel_bank if sel_bank != "All Banks" else True) &
    (summary['Industry'].isin(filtered_inds))
]

# ————————————————————————————————————————————————
# 4) Page title and key metrics
# ————————————————————————————————————————————————
st.title("🏦 Bank Performance Intelligence")
st.markdown(f"**Bank:** `{sel_bank}`   |   **Industries:** {', '.join(filtered_inds)}")

if sel_bank != "All Banks":
    # show metrics cards for selected bank
    st.subheader("🚀 Key Performance Indicators")
    kpis = summary_f.mean()
    c1, c2, c3, c4 = st.columns(4)
    c1.metric("Avg YoY Growth %", f"{kpis['Avg YoY %']:.2f}")
    c2.metric("CAGR % (2020–2024)", f"{kpis['CAGR %']:.2f}")
    c3.metric("Avg Rank", f"{kpis['Avg Rank']:.2f}")
    c4.metric("Rank Volatility (σ)", f"{kpis['Rank Volatility']:.2f}")

# ————————————————————————————————————————————————
# 5) Main analysis tabs
# ————————————————————————————————————————————————
tab1, tab2, tab3 = st.tabs(["Overview", "Trends", "Industry Heatmap"])

with tab1:
    st.subheader("📊 Summary Table")
    st.dataframe(summary_f, use_container_width=True)

with tab2:
    st.subheader("📈 Time-Series Trends")
    # Layout: two charts side-by-side
    t1, t2 = st.columns(2)
    # Net Revenue over time
    fig1 = px.line(df_f, x='Year', y='Net Revenue (Millions USD)',
                   color='Industry', markers=True,
                   title="Revenue Over Time")
    fig1.update_layout(yaxis_title="Millions USD", title_x=0.5)
    t1.plotly_chart(fig1, use_container_width=True)

    # YoY Growth over time
    fig2 = px.bar(df_f, x='Year', y='YoY Growth %',
                  color='Industry', barmode='group',
                  title="Year-Over-Year Growth %")
    fig2.update_layout(title_x=0.5, yaxis_title="Growth %", xaxis=dict(dtick=1))
    t2.plotly_chart(fig2, use_container_width=True)

with tab3:
    st.subheader("🔍 YoY Growth Heatmap")
    # pivot for heatmap: Industries vs Years
    heat = df_f.pivot_table(index='Industry', columns='Year',
                            values='YoY Growth %').loc[filtered_inds]
    fig3 = go.Figure(data=go.Heatmap(
        z=heat.values,
        x=heat.columns.astype(str),
        y=heat.index,
        coloraxis="coloraxis"
    ))
    fig3.update_layout(
        coloraxis=dict(colorscale="Viridis"),
        title="YoY Growth % by Industry & Year",
        xaxis_title="Year",
        yaxis_title="Industry",
        title_x=0.5
    )
    st.plotly_chart(fig3, use_container_width=True)

# ————————————————————————————————————————————————
# Footer
# ————————————————————————————————————————————————
st.markdown("---")
st.markdown(
    "⚙️ Built with Streamlit • Select different banks or industries to "
    "drive all analyses dynamically."
)


2025-04-27 02:35:38.596 No runtime found, using MemoryCacheStorageManager


DeltaGenerator()